In [1]:
from tensorflow.compat.v1 import ConfigProto
from tensorflow.compat.v1 import InteractiveSession

config = ConfigProto()
config.gpu_options.per_process_gpu_memory_fraction = 0.5
config.gpu_options.allow_growth = True
session = InteractiveSession(config=config)

In [2]:
import numpy as np
import matplotlib.pyplot as plt
from glob import glob

In [3]:
# import the libraries as shown below

from tensorflow.keras.layers import Input, Lambda, Dense, Flatten
from tensorflow.keras.models import Model
from tensorflow.keras.applications.vgg19 import VGG19
from tensorflow.keras.applications.vgg19 import preprocess_input
from tensorflow.keras.preprocessing import image
from tensorflow.keras.preprocessing.image import ImageDataGenerator,load_img
from tensorflow.keras.models import Sequential
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping

In [4]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [5]:
# reshape image
IMAGE_SIZE = [224, 224]

train_path = '/content/drive/MyDrive/cnn dataset for project/train'
valid_path = '/content/drive/MyDrive/cnn dataset for project/val'


In [6]:
VGG19  = VGG19 (input_shape=IMAGE_SIZE + [3]
                        , weights='imagenet', include_top=False)

80142336/80134624 [==============================] - 0s 0us/step


In [7]:
#we dont train weights of InceptionV3 layer we already download weights 
for layer in VGG19.layers:
    layer.trainable = False

In [8]:
# to check how many class are present 
folders = glob('/content/drive/MyDrive/cnn dataset for project/train/*')

In [9]:
# create flatten layer of output InceptionV3
x = Flatten()(VGG19.output)

In [10]:
prediction = Dense(len(folders), activation='ReLU')(x)

# create a model 
model = Model(inputs=VGG19.input, outputs=prediction)

In [11]:

# view the structure of the model
model.summary()

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 224, 224, 3)]     0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0     

In [12]:
# tell the model what cost and optimization method to use
model.compile(
  loss='binary_crossentropy',
  optimizer='adam',
  metrics=['accuracy']
)

In [13]:
# Use the Image Data Generator to import the images from the dataset
from tensorflow.keras.preprocessing.image import ImageDataGenerator

train_datagen = ImageDataGenerator(rescale = 1./255,
                                   shear_range = 0.2,
                                   zoom_range = 0.2,
                                   horizontal_flip = True)

test_datagen = ImageDataGenerator(rescale = 1./255)

In [14]:
# Make sure you provide the same target size as initialied for the image size
training_set = train_datagen.flow_from_directory('/content/drive/MyDrive/cnn dataset for project/train',
                                                 target_size = (224, 224),
                                                 batch_size = 32,
                                                 class_mode = 'binary')

Found 2297 images belonging to 2 classes.


In [15]:
test_set = test_datagen.flow_from_directory('/content/drive/MyDrive/cnn dataset for project/val',
                                            target_size = (224, 224),
                                            batch_size = 32,
                                            class_mode = 'binary')

Found 995 images belonging to 2 classes.


In [ ]:
# fit the model
r = model.fit_generator(
  training_set,
  validation_data=test_set,
  epochs=20,
  steps_per_epoch=len(training_set),
  validation_steps=len(test_set)
  )

/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/engine/training.py:1940: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  warnings.warn('`Model.fit_generator` is deprecated and '


Epoch 1/20
144/144 [==============================] - 1093s 7s/step - loss: 7.6531 - accuracy: 0.5816 - val_loss: 7.6761 - val_accuracy: 0.5859
Epoch 2/20
144/144 [==============================] - ETA: 0s - loss: 7.6759 - accuracy: 0.5834

In [ ]:
# plot the loss
plt.plot(r.history['loss'], label='train loss')
plt.plot(r.history['val_loss'], label='val loss')
plt.legend()
plt.show()
plt.savefig('LossVal_loss')

# plot the accuracy
plt.plot(r.history['accuracy'], label='train acc')
plt.plot(r.history['val_accuracy'], label='val acc')
plt.legend()
plt.show()
plt.savefig('AccVal_acc')

In [ ]:
from tensorflow.keras.models import load_model

model.save('model_VGG19.h5')

In [ ]:
y_pred = model.predict(test_set)


In [ ]:
y_pred

In [ ]:
model=load_model('model_VGG19.h5')

In [ ]:
test_image = image.load_img('/content/drive/MyDrive/cnn dataset for project/test/Converted images01651 (13)(1).jpg', target_size = (224,224))
test_image = image.img_to_array(test_image)
test_image=test_image/255
test_image = np.expand_dims(test_image, axis = 0)
img_data=preprocess_input(test_image)
img_data.shape

In [ ]:
model.predict(img_data)


In [ ]:
a=np.argmax(model.predict(img_data), axis=1)

In [ ]:
a

In [ ]:
if(a==0):
    print("Non working")
else:
    print("Working")